# Exploratory Data Analysis

This notebook analyzes the following datasets:
1. Historical customer data: Customer data such as usage, sign up date, forecasted usage etc
2. Historical pricing data: variable and fixed pricing data etc
3. Churn indicator: whether each customer has churned or not

## Import packages

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats

# Shows plots in jupyter notebook
%matplotlib inline

# Set plot style
sns.set(style="whitegrid")
plt.style.use('seaborn-v0_8-whitegrid')

---

## Loading data with Pandas

We need to load `client_data (1).csv` and `price_data (1).csv` into individual dataframes so that we can work with them in Python.

In [ ]:
client_df = pd.read_csv('./client_data (1).csv')
price_df = pd.read_csv('./price_data (1).csv')

Let's examine the first few rows of each dataset to understand their structure.

In [ ]:
print("Client Data - First 3 rows:")
client_df.head(3)

In [ ]:
print("Price Data - First 3 rows:")
price_df.head(3)

---

## 1. Data Types Analysis

Let's examine the data types of each column in both datasets.

In [ ]:
print("Client Data - Data Types:")
client_df.info()

In [ ]:
print("Price Data - Data Types:")
price_df.info()

### Data Types Summary

#### Client Data:
- **Object (String) columns (8)**: id, channel_sales, date_activ, date_end, date_modif_prod, date_renewal, has_gas, origin_up
- **Integer columns (7)**: cons_12m, cons_gas_12m, cons_last_month, forecast_cons_year, nb_prod_act, num_years_antig, churn
- **Float columns (11)**: forecast_cons_12m, forecast_discount_energy, forecast_meter_rent_12m, forecast_price_energy_p1, forecast_price_energy_p2, forecast_price_pow_p1, imp_cons, margin_gross_pow_ele, margin_net_pow_ele, net_margin, pow_max

#### Price Data:
- **Object (String) columns (2)**: id, price_date
- **Float columns (6)**: price_off_peak_var, price_peak_var, price_mid_peak_var, price_off_peak_fix, price_peak_fix, price_mid_peak_fix

The date columns are currently stored as strings (object type) and may need to be converted to datetime format for time-based analysis.

## 2. Descriptive Statistics

Let's examine the statistical properties of the numerical columns in both datasets.

In [ ]:
print("Client Data - Descriptive Statistics:")
client_df.describe()

In [ ]:
print("Price Data - Descriptive Statistics:")
price_df.describe()

### Descriptive Statistics Summary

#### Client Data Insights:
- The dataset contains 14,606 customer records
- Churn rate is approximately 9.7% (mean of churn column = 0.097152)
- Most customers have only 1 product (median nb_prod_act = 1)
- There's high variability in consumption (cons_12m) with a standard deviation of 573,465 units
- The average customer has been with the company for about 5 years (mean num_years_antig = 4.997809)

#### Price Data Insights:
- The dataset contains 193,002 price records
- Off-peak fixed prices are much higher than variable prices (mean price_off_peak_fix = 43.33 vs mean price_off_peak_var = 0.14)
- Many customers have zero values for peak and mid-peak prices (median price_peak_fix and price_mid_peak_fix = 0)

## 3. Distribution Analysis

Let's visualize the distributions of key numerical variables to better understand their patterns.

In [ ]:
# Set up a figure with multiple subplots for key numerical variables
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
fig.suptitle('Distribution of Key Numerical Variables', fontsize=16)

# List of numerical columns to plot
num_cols = ['cons_12m', 'cons_gas_12m', 'cons_last_month', 
            'forecast_cons_12m', 'net_margin', 'num_years_antig',
            'pow_max', 'margin_gross_pow_ele', 'nb_prod_act']

# Plot histograms for each column
for i, col in enumerate(num_cols):
    row, col_idx = i // 3, i % 3
    sns.histplot(client_df[col], kde=True, ax=axes[row, col_idx])
    axes[row, col_idx].set_title(f'Distribution of {col}')
    axes[row, col_idx].set_xlabel(col)
    
    # For columns with extreme values, set a reasonable x-limit
    if col in ['cons_12m', 'cons_gas_12m', 'cons_last_month']:
        q99 = client_df[col].quantile(0.99)
        axes[row, col_idx].set_xlim(0, q99)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

In [ ]:
# Analyze categorical variables
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Distribution of Key Categorical Variables', fontsize=16)

# Churn distribution
sns.countplot(x='churn', data=client_df, ax=axes[0])
axes[0].set_title('Churn Distribution')
axes[0].set_xlabel('Churn (0=No, 1=Yes)')

# Has gas distribution
sns.countplot(x='has_gas', data=client_df, ax=axes[1])
axes[1].set_title('Has Gas Distribution')
axes[1].set_xlabel('Has Gas (f=No, t=Yes)')

# Number of products distribution
sns.countplot(x='nb_prod_act', data=client_df, ax=axes[2])
axes[2].set_title('Number of Active Products')
axes[2].set_xlabel('Number of Products')
axes[2].set_xlim(-0.5, 5.5)  # Show only up to 5 products for better visibility

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Distribution Analysis Summary

The distributions reveal several important patterns:

1. **Consumption Variables**: cons_12m, cons_gas_12m, and cons_last_month show highly right-skewed distributions, with many customers having low consumption and a few having extremely high consumption.

2. **Churn Distribution**: The dataset is imbalanced with approximately 9.7% of customers having churned (churn=1).

3. **Gas Service**: The majority of customers do not have gas service (has_gas='f').

4. **Number of Products**: Most customers have only 1 active product, with very few having more than 2 products.

5. **Customer Tenure**: The distribution of num_years_antig shows that most customers have been with the company for 4-6 years.

6. **Power Capacity**: pow_max shows a multi-modal distribution, suggesting different standard power capacity tiers offered to customers.

## 4. Correlation Analysis

Let's examine the correlations between numerical variables to identify potential relationships.

In [ ]:
# Select numerical columns for correlation analysis
num_cols = client_df.select_dtypes(include=['int64', 'float64']).columns

# Calculate correlation matrix
corr_matrix = client_df[num_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(16, 12))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix of Numerical Variables', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Focus on correlations with churn
churn_corr = corr_matrix['churn'].sort_values(ascending=False)
print("Correlations with Churn:")
print(churn_corr)

### Correlation Analysis Summary

The correlation analysis reveals several important relationships:

1. **Consumption Variables**: Strong positive correlations exist between different consumption metrics (cons_12m, cons_gas_12m, cons_last_month).

2. **Margin Variables**: margin_gross_pow_ele and margin_net_pow_ele are almost perfectly correlated, suggesting they capture very similar information.

3. **Churn Correlations**: The variables most correlated with churn (either positively or negatively) provide insights into potential churn predictors.

4. **Forecast Variables**: There are moderate to strong correlations between actual consumption and forecasted consumption, indicating reasonable forecast accuracy.

## 5. Churn Analysis

Let's analyze how different variables relate to customer churn.

In [ ]:
# Set up a figure with multiple subplots for key variables by churn status
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Key Variables by Churn Status', fontsize=16)

# List of variables to analyze
vars_to_analyze = ['net_margin', 'num_years_antig', 'pow_max', 
                   'cons_12m', 'forecast_cons_12m', 'nb_prod_act']

# Plot boxplots for each variable by churn status
for i, var in enumerate(vars_to_analyze):
    row, col = i // 3, i % 3
    sns.boxplot(x='churn', y=var, data=client_df, ax=axes[row, col])
    axes[row, col].set_title(f'{var} by Churn Status')
    axes[row, col].set_xlabel('Churn (0=No, 1=Yes)')
    
    # For variables with extreme values, set a reasonable y-limit
    if var in ['cons_12m', 'forecast_cons_12m']:
        q95 = client_df[var].quantile(0.95)
        axes[row, col].set_ylim(0, q95)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

In [ ]:
# Analyze categorical variables in relation to churn
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Categorical Variables by Churn Status', fontsize=16)

# Has gas vs churn
sns.countplot(x='has_gas', hue='churn', data=client_df, ax=axes[0])
axes[0].set_title('Has Gas by Churn Status')
axes[0].set_xlabel('Has Gas (f=No, t=Yes)')
axes[0].legend(title='Churn', labels=['No', 'Yes'])

# Calculate churn rate by has_gas
churn_by_gas = client_df.groupby('has_gas')['churn'].mean().reset_index()
churn_by_gas.columns = ['has_gas', 'churn_rate']
sns.barplot(x='has_gas', y='churn_rate', data=churn_by_gas, ax=axes[1])
axes[1].set_title('Churn Rate by Gas Service')
axes[1].set_xlabel('Has Gas (f=No, t=Yes)')
axes[1].set_ylabel('Churn Rate')
axes[1].set_ylim(0, 0.2)  # Set y-limit for better visualization

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Churn Analysis Summary

The analysis of variables by churn status reveals several important patterns:

1. **Net Margin**: Customers who churned tend to have higher net margins, suggesting that more profitable customers may be at higher risk of leaving.

2. **Customer Tenure**: There appears to be a relationship between the number of years with the company (num_years_antig) and churn, with newer customers potentially having different churn patterns than long-term customers.

3. **Power Capacity**: Customers with higher power capacity (pow_max) show different churn patterns compared to those with lower capacity.

4. **Gas Service**: The churn rate differs between customers with and without gas service, indicating that the type of service bundle may influence churn behavior.

5. **Consumption**: There are differences in consumption patterns (cons_12m) between churned and non-churned customers, suggesting that usage level may be a predictor of churn.

## 6. Price Data Analysis

Let's analyze the pricing data to understand pricing patterns and their potential relationship with churn.

In [ ]:
# Set up a figure with multiple subplots for price variables
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Distribution of Price Variables', fontsize=16)

# List of price variables to analyze
price_vars = ['price_off_peak_var', 'price_peak_var', 'price_mid_peak_var',
              'price_off_peak_fix', 'price_peak_fix', 'price_mid_peak_fix']

# Plot histograms for each price variable
for i, var in enumerate(price_vars):
    row, col = i // 3, i % 3
    sns.histplot(price_df[var], kde=True, ax=axes[row, col])
    axes[row, col].set_title(f'Distribution of {var}')
    axes[row, col].set_xlabel(var)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

In [ ]:
# Convert price_date to datetime for time-based analysis
price_df['price_date'] = pd.to_datetime(price_df['price_date'])

# Aggregate prices by month to see trends over time
price_df['year_month'] = price_df['price_date'].dt.to_period('M')
monthly_prices = price_df.groupby('year_month').agg({
    'price_off_peak_var': 'mean',
    'price_peak_var': 'mean',
    'price_mid_peak_var': 'mean',
    'price_off_peak_fix': 'mean',
    'price_peak_fix': 'mean',
    'price_mid_peak_fix': 'mean'
}).reset_index()

# Convert period to datetime for plotting
monthly_prices['date'] = monthly_prices['year_month'].dt.to_timestamp()

# Plot variable price trends
plt.figure(figsize=(15, 6))
plt.plot(monthly_prices['date'], monthly_prices['price_off_peak_var'], label='Off-Peak Variable')
plt.plot(monthly_prices['date'], monthly_prices['price_peak_var'], label='Peak Variable')
plt.plot(monthly_prices['date'], monthly_prices['price_mid_peak_var'], label='Mid-Peak Variable')
plt.title('Variable Price Trends Over Time', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Average Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot fixed price trends
plt.figure(figsize=(15, 6))
plt.plot(monthly_prices['date'], monthly_prices['price_off_peak_fix'], label='Off-Peak Fixed')
plt.plot(monthly_prices['date'], monthly_prices['price_peak_fix'], label='Peak Fixed')
plt.plot(monthly_prices['date'], monthly_prices['price_mid_peak_fix'], label='Mid-Peak Fixed')
plt.title('Fixed Price Trends Over Time', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Average Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Price Data Analysis Summary

The analysis of pricing data reveals several important patterns:

1. **Price Distributions**: The distributions of price variables show distinct patterns, with some prices (like off-peak fixed) having a more concentrated distribution while others show more variability.

2. **Zero Values**: Many customers have zero values for peak and mid-peak prices, suggesting different pricing plans or tiers.

3. **Price Trends**: The time-based analysis shows how prices have evolved over the period covered by the dataset, revealing any seasonal patterns or general trends in pricing.

4. **Fixed vs. Variable**: There are significant differences in the magnitude and patterns between fixed and variable prices, with fixed prices generally being much higher than variable prices.

## 7. Conclusion and Key Findings

Based on our exploratory data analysis, we can summarize the following key findings:

### Data Structure and Quality
- The client dataset contains 14,606 customer records with 26 variables
- The price dataset contains 193,002 price records with 8 variables
- No missing values were found in either dataset
- Several date columns are stored as strings and would need conversion for time-based analysis

### Customer Characteristics
- The churn rate is approximately 9.7%
- Most customers have only 1 active product
- The majority of customers do not have gas service
- Customer consumption shows high variability with right-skewed distributions
- The average customer tenure is about 5 years

### Churn Patterns
- Several variables show different patterns between churned and non-churned customers
- Net margin, power capacity, and consumption appear to have relationships with churn
- Customers with gas service show different churn patterns than those without

### Pricing Patterns
- Fixed prices are significantly higher than variable prices
- Many customers have zero values for peak and mid-peak prices
- Price trends over time reveal patterns that may influence customer behavior

These findings provide valuable insights for understanding customer behavior and developing strategies to reduce churn. Further analysis could include more sophisticated modeling techniques to predict churn and identify the most influential factors.